# Определение цвета товара по фотографии

## Предварительные требования (Prerequisites)

Для запуска этого ноутбука требуются следующие зависимости:

```
Python >= 3.10
numpy >= 1.24.3
torch >= 2.0.1
torchvision >= 0.15.2
timm >= 0.9.7
Pillow >= 10.0.0
```

Вы можете установить их с помощью pip:
```bash
pip install numpy torch torchvision timm Pillow
```

Перед использованием убедитесь, что у вас есть файл с весами модели.

In [1]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
import timm
import torchvision.transforms as transforms

# Словари соответствия
TRANSLIT_TO_RU = {
    'bezhevyi': 'бежевый',
    'belyi': 'белый',
    'biryuzovyi': 'бирюзовый',
    'bordovyi': 'бордовый',
    'goluboi': 'голубой',
    'zheltyi': 'желтый',
    'zelenyi': 'зеленый',
    'zolotoi': 'золотой',
    'korichnevyi': 'коричневый',
    'krasnyi': 'красный',
    'oranzhevyi': 'оранжевый',
    'raznocvetnyi': 'разноцветный',
    'rozovyi': 'розовый',
    'serebristyi': 'серебряный',
    'seryi': 'серый',
    'sinii': 'синий',
    'fioletovyi': 'фиолетовый',
    'chernyi': 'черный'
}

# Создаем обратное соответствие с русского на транслитерацию
RU_TO_TRANSLIT = {v: k for k, v in TRANSLIT_TO_RU.items()}

# Словарь цветов
COLORS = {
    'бежевый': 'beige',
    'белый': 'white',
    'бирюзовый': 'turquoise',
    'бордовый': 'burgundy',
    'голубой': 'blue',
    'желтый': 'yellow',
    'зеленый': 'green',
    'золотой': 'gold',
    'коричневый': 'brown',
    'красный': 'red',
    'оранжевый': 'orange',
    'разноцветный': 'variegated',
    'розовый': 'pink',
    'серебряный': 'silver',
    'серый': 'gray',
    'синий': 'blue',
    'фиолетовый': 'purple',
    'черный': 'black'
}

# Категории
CATEGORIES = ['одежда для девочек', 'столы', 'стулья', 'сумки']

# Глобальная переменная для хранения загруженной модели
MODEL = None
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ColorClassifier(nn.Module):
    def __init__(self, num_colors, num_categories):
        super().__init__()
        # Используем более легкий и быстрый вариант ViT
        self.backbone = timm.create_model(
            'beitv2_large_patch16_224', 
            pretrained=True, 
            num_classes=0,  # Без верхнего слоя классификации
        )
        
        # Фиксируем большую часть весов для ускорения обучения
        for param in list(self.backbone.parameters())[:-30]:
            param.requires_grad = False
            
        # Расширение для быстрой инференции с кэшированием
        self.backbone.reset_classifier(0)
        
        # Размерность признаков модели
        self.feature_dim = self.backbone.embed_dim
        
        # Эмбеддинг категории
        self.category_embedding = nn.Embedding(num_categories, 32)
        
        # Классификационная голова
        self.classifier = nn.Sequential(
            nn.Linear(self.feature_dim + 32, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_colors)
        )
        
        # Для оптимизации torch.jit
        self.example_input = torch.zeros(1, 3, 224, 224)
        self.example_category = torch.LongTensor([0])
        
    def forward(self, x, category):
        features = self.backbone(x)
        
        category_emb = self.category_embedding(category)
        combined = torch.cat([features, category_emb], dim=1)
        
        return self.classifier(combined)

def load_model(model_path):
    """
    Загружает ранее обученную модель из указанного пути.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    try:
        # Пробуем загрузить как TorchScript модель
        model = torch.jit.load(model_path, map_location=device)
        print("Загружена оптимизированная TorchScript модель")
        return model
    except:
        # Загружаем как обычную модель
        print("Загрузка модели из стандартных весов...")
        model = ColorClassifier(len(COLORS), len(CATEGORIES))
        
        checkpoint = torch.load(model_path, map_location=device)
        if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['model_state_dict'])
        else:
            model.load_state_dict(checkpoint)
        
        # Установка модели в режим оценки
        model.eval()
        model = model.to(device)
        
        return model

def initialize_model(model_path="vit_color_classifier.pth"):
    """
    Инициализирует модель один раз и сохраняет её в глобальной переменной.
    Эта функция должна быть вызвана один раз в начале вашего приложения.
    
    Args:
        model_path (str): Путь к весам модели
        
    Returns:
        Загруженная модель
    """
    global MODEL
    if MODEL is None:
        print("Загрузка модели в первый раз...")
        MODEL = load_model(model_path)
    else:
        print("Модель уже загружена, повторное использование...")
    
    return MODEL

def predict_color(image_path, category_name):
    """
    Предсказывает цвет товара по изображению и его категории.
    Использует глобально загруженную модель (убедитесь, что вызвали initialize_model сначала).
    
    Args:
        image_path (str): Путь к изображению товара
        category_name (str): Название категории товара (должно быть одним из CATEGORIES)
        
    Returns:
        tuple: (best_color, top5_colors) где:
            - best_color (str): Наиболее вероятный цвет в транслитерированной форме (например, 'bezhevyi')
            - top5_colors (dict): Словарь с топ-5 цветами (в транслитерированной форме) и их вероятностями
    """
    global MODEL, DEVICE
    
    # Проверяем, загружена ли модель
    if MODEL is None:
        raise RuntimeError("Модель не инициализирована. Пожалуйста, вызовите initialize_model() сначала.")
    
    # Проверяем название категории
    if category_name not in CATEGORIES:
        raise ValueError(f"Категория должна быть одной из: {CATEGORIES}")
    
    # Проверяем существование изображения
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Изображение не найдено: {image_path}")
    
    # Подготавливаем трансформацию изображения
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    
    # Загружаем и трансформируем изображение
    try:
        image = Image.open(image_path).convert('RGB')
        image_tensor = transform(image).unsqueeze(0).to(DEVICE)
    except Exception as e:
        raise RuntimeError(f"Ошибка при обработке изображения: {str(e)}")
    
    # Получаем индекс категории
    category_idx = CATEGORIES.index(category_name)
    category_tensor = torch.tensor([category_idx], dtype=torch.long).to(DEVICE)
    
    # Делаем предсказание
    with torch.no_grad():
        outputs = MODEL(image_tensor, category_tensor)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()[0]
    
    # Получаем названия цветов на русском
    color_list = list(COLORS.keys())
    
    # Находим лучший цвет
    best_color_idx = np.argmax(probs)
    best_color_ru = color_list[best_color_idx]
    
    # Преобразуем в транслитерированную форму
    best_color = RU_TO_TRANSLIT[best_color_ru]
    
    # Получаем топ-5 цветов с вероятностями
    top5_indices = np.argsort(probs)[-5:][::-1]
    
    # Преобразуем цвета в транслитерированную форму в результате
    top5_colors = {RU_TO_TRANSLIT[color_list[idx]]: float(probs[idx]) for idx in top5_indices}
    
    return best_color, top5_colors

# Пример использования:
# 1. Инициализируем модель один раз в начале
# model = initialize_model("vit_color_classifier.pth")
#
# 2. Делаем предсказания столько раз, сколько нужно, без повторной загрузки модели
# best_color, top5_colors = predict_color("path/to/image.jpg", "столы")
# print(f"Наиболее вероятный цвет: {best_color}")
# for color, prob in top5_colors.items():
#     print(f"  {color}: {prob:.4f}")
#
# 3. Делаем еще одно предсказание с той же моделью
# best_color2, top5_colors2 = predict_color("path/to/another_image.jpg", "стулья")


ModuleNotFoundError: No module named 'numpy'

In [4]:
model = initialize_model("/kaggle/input/macro_/pytorch/default/1/macro_weights.pth")

Loading model for the first time...
Loading model from standard weights...


<ipython-input-3-c8fbc8292a2c>:123: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


In [5]:
best_color, top5_colors = predict_color("/kaggle/input/colors/dataset_colors/test_data/19762915377.png", "одежда для девочек")
print(f"Наиболее вероятный цвет: {best_color}")  # Теперь возвращает 'bezhevyi' вместо 'бежевый'
print("Топ-5 цветов:")
for color, prob in top5_colors.items():
    print(f"  {color}: {prob:.4f}")  # Цвета теперь в транслитерированной форме


Best color: zelenyi
Top 5 colors:
  zelenyi: 0.9715
  chernyi: 0.0067
  raznocvetnyi: 0.0044
  korichnevyi: 0.0040
  sinii: 0.0034


## Интерактивный пример

Ниже вы можете ввести путь к вашему изображению и выбрать категорию товара для определения его цвета:

In [ ]:
# Интерактивный ввод для тестирования на произвольных изображениях
image_path = input("Введите путь к изображению: ")
print("Доступные категории:")
for i, category in enumerate(CATEGORIES):
    print(f"{i+1}. {category}")
category_idx = int(input("Выберите номер категории (1-4): ")) - 1

if 0 <= category_idx < len(CATEGORIES):
    category = CATEGORIES[category_idx]
    best_color, top5_colors = predict_color(image_path, category)
    
    print(f"\nНаиболее вероятный цвет: {TRANSLIT_TO_RU[best_color]}")
    print("Топ-5 цветов:")
    for color, prob in top5_colors.items():
        print(f"  {TRANSLIT_TO_RU[color]}: {prob:.4f}")
else:
    print("Неверный номер категории.")